# Custom Chatbot Notebook

An OpenAI client is initialised by using environment variables and a tokenizer is set up for a specific model (`gpt-4o-mini-2024-07-18`). Also,the necessary libraries and custom utility functions are imported.

In [31]:
import pandas as pd
import os
from pathlib import Path
import tiktoken

# Custom Functions
from fncs.utilities import (
    create_ollama_client,
    response_generator,
    prompt_builder
    )
from fncs.retrieval import (
    get_embedding,
    search_text,
    control_chunk_context
    )

from fncs.prompt_templates import (
    user_prompt,
    user_prompt_without_context
)

# Deployment model names
chat_name = 'gemma3:1b'
emb_name = 'granite-embedding' # 'nomic-embed-text' # 'granite-embedding'

# Initialising ollama client
ollama_client = create_ollama_client()

# currently I use a gpt-4o tokeniser which is actually wrong
# I will implement an ollama tokeniser in the future
tokenizer = tiktoken.encoding_for_model('gpt-4o')

### Loading dataset

In [32]:
proj_dir = Path(os.getcwd())
df = pd.read_csv(proj_dir / "data" / "2023_fashion_trends_embeddings_ollama.csv")
df.head(3)

,text,embeddings
0,\nFashion trends according to refinery29\n\nSo...,"[-0.099385194, 0.041942384, 0.02769633, -0.036..."
1,\nFashion trends according to refinery29\n\nSo...,"[-0.039359834, 0.079903804, 0.016821878, -0.02..."
2,\nFashion trends according to refinery29\n\nSo...,"[-0.10401276, 0.063805886, 0.03733611, -0.0067..."


The embeddings are stored as text/string in the DataFrame and need to be converted to lists/arrays

In [33]:
import ast
# Converting the string representations of embeddings to actual lists
df['embeddings'] = df['embeddings'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

Checking transformation

In [34]:
type(df[['embeddings']].iloc[0].values[0])

list

### Calculating Cosine Distances based on query


Below I create a query string about fashion trends in 2023. Then, by using the `get_embedding` function, the embeddings of the query are generated, by passing the query, OpenAI client, and embedding model as inputs.

In [35]:
query = "What is the most popular fashion trend about pants in 2023?"
query_emb = get_embedding(text=query, client = ollama_client, model=emb_name)

The DataFrame `df` is sorted based on the cosine distance between the query embedding (`query_emb`) and the embeddings in the DataFrame using the `search_text` function, and stores the result in `df_sorted`.

In [36]:
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

In [37]:
df_sorted.head()

,text,embeddings,distance
1,\nFashion trends according to refinery29\n\nSo...,"[-0.039359834, 0.079903804, 0.016821878, -0.02...",0.127356
44,\nFashion trends according to whowhatwear\n\nS...,"[-0.075561, 0.060711168, 0.008432559, 0.012441...",0.132610
58,\nFashion trends according to whowhatwear\n\nS...,"[-0.063408405, 0.04534362, 0.0113590695, 0.002...",0.133518
45,\nFashion trends according to whowhatwear\n\nS...,"[-0.066249, 0.05083767, -0.011095402, 0.021163...",0.151187
40,\nFashion trends according to whowhatwear\n\nS...,"[-0.088052295, 0.048361346, 0.012177077, 0.011...",0.157143


### Prompt Template



Creating the system prompt to be used in the chatbot

In [38]:
system_prompt = "You are an expert fashion trend analyser. Based only on the provided information you must analyse and summarise the trends and provide an accurate answer."

print(f"System Prompt Tokens: {len(tokenizer.encode(system_prompt))}")

System Prompt Tokens: 28


Calling the user prompt templates functions that will

In [39]:
print(f"User Prompt (with context) Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt()))}")

User Prompt (with context) Tokens BEFORE context insertion: 130


In [40]:
# to be used in performance demonstration later
print(f"User Prompt (without context) Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt_without_context()))}")

User Prompt (without context) Tokens BEFORE context insertion: 94


#### Apply token controller function ( fnc: control_chunk_context )

The variable `max_token_count` to 1000, serves as a limit for the total number of tokens allowed in a prompt.

In [41]:
#parameter that control the prompt tokens:
max_token_count = 1000

The code below calculates the current token count of the prompts (system and user) and generates a context by selecting data from the sorted DataFrame (`df_sorted`) based on a maximum allowed token limit (`max_token_count`) using the `control_chunk_context` function.

In [42]:
current_token_count = len(tokenizer.encode(user_prompt())) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(
    df_sorted,
    current_token_count,
    max_token_count,
    tokenizer = tokenizer
)

 Below, the final `user_prompt` is created by inserting the generated `context` into the prompt template and by formatting it with the query and context.

In [43]:
# prompt template params
context_inprompt = "\n----\n".join(context)
user_prompt_0 = user_prompt().format(query, context_inprompt)

print(user_prompt_0)


    ***Question: What is the most popular fashion trend about pants in 2023?
    
    ***Context:
    <--Start of Context-->
    
Fashion trends according to refinery29

Source Title: 7 Fashion Trends That Will Take Over 2023 — Shop Them Now

2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, this trend is translated into tailored silhouettes, interesting pocket placements, elevated fabrics like silk and organza, and colors that go beyond khaki and olive.

Source URL: https://www.refinery29.com/en-us/fashion-trends-2023

----

Fashion trends according to whowhatwear

Source Title: Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See

I get it. Some of the trends on this list might not translate seamlessly into everyday life (if you're prepared to wear a completely sheer skirt to run errands in, more power to you). Howev

In [44]:
print(f"User Prompt Tokens AFTER context insertion: {len(tokenizer.encode(user_prompt_0))}")

User Prompt Tokens AFTER context insertion: 839


## Custom Query Completion

**Finally, the code below generates a final prompt using the `prompt_builder` function by combining the system and user prompts. It then sends the prompt to the OpenAI model (`chat_model`) using the `response_generator` function with specified additional options (e.g., `temperature=0.1`) to generate an AI response. It also calculates the total cost in EUR based on the API usage (`response_full.usage`) for the specific deployment (`gpt-4o`).**

In [45]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_0)
additional_options = {"temperature": 0.1,}
response, response_full = response_generator(ollama_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)


In [46]:
print(response)

1. Answer: Cargo pants are currently the most popular fashion trend regarding pants in 2023, appearing in various silhouettes and colors across multiple sources.

2. Sources:
   • Refinery29: https://www.refinery29.com/en-us/fashion-trends-2023
   • Who What Wear: https://www.whowhatwear.com/spring-summer-2023-fashion-trends/


In [47]:
print('Total Tokens: ', response_full.usage.total_tokens)
print('Total Completion Tokens: ', response_full.usage.completion_tokens)
print('Total Prompt Tokens: ', response_full.usage.prompt_tokens)

Total Tokens:  1083
Total Completion Tokens:  99
Total Prompt Tokens:  984


## Demonstrating Performance

Below I demonstrate through some examples how using a custom prompt significantly enhances the performance and accuracy of responses from the OpenAI model compared to basic (generic) prompts. I showcase two different example queries about 2023 fashion trends, providing the responses produced using the custom context-based prompt and a basic prompt (without context).

In other words, the comparison made below, clearly illustrates that custom prompts, enriched with relevant contextual information, greatly enhance the accuracy, specificity, and usefulness of model-generated responses. Basic prompts, without contextual enrichment, yield generic and less informative answers.

### Question 1
**Question**: According to Vogue, what is a new trend presented by Prada on New York Fashion Week?

In [48]:
query_1 = "According to Vogue, what is a new trend presented by Prada on New York Fashion Week?"
max_token_count = 1000

1.1 Using custom prompt with context:

In [49]:
query_emb = get_embedding(text=query_1, client = ollama_client, model=emb_name)
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

current_token_count = len(tokenizer.encode(user_prompt())) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(chunks_sorted_df=df_sorted,
                                current_token_count=current_token_count,
                                max_token_count=max_token_count,
                                tokenizer = tokenizer)
context_inprompt = "\n----\n".join(context)
user_prompt_1 = user_prompt().format(query_1, context_inprompt)

final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_1)
additional_options = {"temperature": 0.1,}

response_1_1, response_full_1_1 = \
    response_generator(ollama_client, chat_model=chat_name, prompts=final_prompt, options= additional_options)

In [50]:
print(response_1_1)

**Answer:**

 Prada’s new trend involves satin midi skirts with an irregularly dyed print and a slit that’s deliberately torn, evoking a sense of unfinishedness. Vogue’s editors are also incorporating lingerie-esque going-out looks featuring sheer and lace fabrics.  The brand is also showcasing cropped leather pieces, particularly from Musier Paris, and Prada’s Moon bag as key accessories.  The trend also includes cinched waist silhouettes, inspired by biker jackets and the neo-minimalism of Khaite, The Row, and Peter Do.  Finally, the trend leans into a tailored look with pared-back silhouettes, incorporating button-downs, knits, and slightly unbuttoned blazers.

**Sources:**

*   https://www.vogue.com/article/spring-2023-trends-editors-picks
*   https://www.vogue.com/article/spring-2023-trends-editors-picks
*   https://www.vogue.com/article/spring-2023-trends-editors-picks
*   https://www.vogue.com/article/spring-2023-trends-editors-picks
*   https://www.vogue.com/article/spring-2023

1.2 Using basic prompt with no context:

In [51]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_without_context().format(query_1) )
additional_options = {"temperature": 0.1,}

response_1_2, response_full_1_2 = response_generator(ollama_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)

In [52]:
print(response_1_2)

Cannot be determined as I do not have enough the knowledge to answer this question.


### Question 2
**Question**: According to glamour magazine and whowhatwear.com what are the denim fashion trends for the year 2023?

In [53]:
query_2 = "According to glamour magazine and whowhatwear.com what are the denim fashion trends for the year 2023?"
max_token_count = 1000

2.1 Using custom prompt with context:

In [54]:
query_emb = get_embedding(text=query_2, client = ollama_client, model=emb_name)
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

current_token_count = len(tokenizer.encode(user_prompt())) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(chunks_sorted_df=df_sorted,
                                current_token_count=current_token_count,
                                max_token_count=max_token_count,
                                tokenizer = tokenizer)
context_inprompt = "\n----\n".join(context)
user_prompt_2 = user_prompt().format(query_2, context_inprompt)

final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_2)
additional_options = {"temperature": 0,}

response_2_1, response_full_2_1 = \
    response_generator(ollama_client, chat_model=chat_name, prompts=final_prompt, options= additional_options)

In [55]:
print(response_2_1)

**Answer:**

Denim is experiencing a significant resurgence, moving beyond traditional styles and embracing looser silhouettes. From double-waisted jeans to carpenter jeans, there’s a wide range of cuts and washes available.  The trend leans towards timeless silhouettes that can be easily incorporated into various outfits.  Specifically, wide-leg trousers, particularly from brands like Altuzarra and Bally, are expected to remain prominent.  The focus is on relaxed, comfortable styles that feel effortlessly chic.  The influence of post-lockdown fashion is evident in a desire for more relaxed silhouettes, with a return to comfortable, easy-to-wear clothing.

**Sources:**

• https://www.refinery29.com/en-us/fashion-trends-2023
• https://www.glamour.com/story/spring-fashion-trends
• https://www.vogue.com/article/spring-2023-trends-editors-picks
• https://www.whowhatwear.com/spring-summer-2023-fashion-trends/


2.2 Using basic prompt with no context:

In [56]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_without_context().format(query_2) )
additional_options = {"temperature": 0,}

response_2_2, response_full_2_2 = response_generator(ollama_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)

In [57]:
print(response_2_2)

Okay, here’s my analysis based solely on Glamour and Who What Wear for the denim trends of 2023:

**Answer:**

Denim is experiencing a significant shift, moving away from the overly distressed, chunky aesthetic of previous years.  The dominant trends for 2023 are a return to classic, slightly oversized washes, particularly in washes of indigo and dark wash.  A key element is the incorporation of subtle distressing – a gentle, almost artistic, fraying – rather than aggressive rips.  Furthermore, there’s a focus on high-quality denim, with a preference for heavier weight denim and a luxurious feel.  A subtle, almost retro influence is also present, with a nod to the 70s and 80s denim styles.  Finally, the denim is being paired with more refined, neutral-toned pieces like cashmere sweaters and slip dresses.

**Sources:**

*   **Glamour:** [https://www.glamour.com/fashion/trends/denim-trends-2023](https://www.glamour.com/fashion/trends/denim-trends-2023)
*   **Who What Wear:** [https://www

In [58]:
print('Notebook Finished')

Notebook Finished
